In [2]:
# TPU Library Installation
# !pip install torch-xla

!pip install torchviz
!pip install torchview
!pip install torchsummary

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=2cdf479a39352a849a6287e72927f191056c80316731051b188fdab8af467163
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [3]:
# Continue with regular imports
import matplotlib.pyplot as plt

import torch
import torchvision

try : 
    
    # imports the torch_xla package
    import torch_xla
    import torch_xla.core.xla_model as xm
except : 
    pass

from PIL import Image
from torch import nn
from torchvision import transforms,datasets
from torch.utils.data import DataLoader

# to visualize model Architecture
from torchview import draw_graph

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    %pip install -q torchinfo
    from torchinfo import summary
CUDA_LAUNCH_BLOCKING=1

torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
train_dir = "/kaggle/input/jaher-notice-4th-time-dataset-with-1917-train-img/Jaher Notice full dataset/Train Data"
test_dir = "/kaggle/input/jaher-notice-4th-time-dataset-with-1917-train-img/Jaher Notice full dataset/Test Data"

In [5]:
NUM_WORKERS = 8

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):
  
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

## Auto `transforms`

In [6]:
# Get a set of pretrained model weights
weights = torchvision.models.EfficientNet_B4_Weights.DEFAULT # .DEFAULT = best available weights from pretraining on ImageNet
weights

EfficientNet_B4_Weights.IMAGENET1K_V1

In [7]:
# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[380]
    resize_size=[384]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [8]:
# This Technique is not worked very well (31-May-2024)

# # Define additional augmentation transformations
# color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)

# # Combine both transformations
# customized_transforms = transforms.Compose([
#     auto_transforms,
#     color_jitter,
# ])

In [9]:
# Create training and testing DataLoaders as well as get a list of class names
train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=auto_transforms, # perform same data transforms on our own data as the pretrained model
                                                                               batch_size=32) # set mini-batch size to 32

train_dataloader, test_dataloader, class_names

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


(<torch.utils.data.dataloader.DataLoader at 0x7bc07ed14e50>,
 ['JN', 'dump'])

In [10]:
# choose this to train on new dataset
model = torchvision.models.efficientnet_b4(weights=weights).to(device)

# choose this to finetune on new similar data that it has already been trained on.
# model = torch.load('/kaggle/input/jaher_notice_segregator_base_model_pytorch/pytorch/base_model/1/full_model_model_train_loss_ 0.0460 _ train_acc_ 0.5326 _ test_loss_ 84.5817 _ test_acc_ 0.9679.pt')

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:01<00:00, 53.9MB/s]


# *freezing only pre-trained model parameters and allowing externally added classification layer weights to be updated*

In [11]:
for name,param in model.features[:8].named_parameters():
    param.requires_grad = False

In [12]:
# for name,param in model.features.named_parameters():
#     print(name,",        req_grad = ", param.requires_grad)

In [13]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
# for param in model.parameters():
#     param.requires_grad = False

In [14]:
# # Allow All the classifier layer parameter to be updated during training
# for name,param in model.classifier.named_parameters():
# #     print(name)
#     param.requires_grad = True

In [15]:
# EXPERIMENTAL CLASSIFIER ARCHITECTURE
# (added addtional 64 node dense layer after 128 node dense layer with ReLu Activation Function)

import torch
import torch.nn as nn
import torch.nn.init as init

# "He/Kaiming Normal" Weight Initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

# Assuming 'device' is already defined

# Recreate the classifier layer and seed it to the target device
''' normalization added after every linear layer. "64,32,16 neuron dense layer also added.",
"dropout layers also added, He weight initializer also added."
'''

model.classifier = nn.Sequential(
    nn.Linear(in_features=1792, out_features=1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(p=0.2),  # Dropout added after the first linear layer
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(p=0.3),  # Dropout added after the second linear layer
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(p=0.4),  # Dropout added after the third linear layer
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1, bias=True)  # Final output layer
).to(device)

# Apply He initialization to all linear layers
model.apply(init_weights)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [16]:
# Define loss and optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

In [17]:
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)
      # y_pred = torch.sigmoid(model(X))
      # 2. Calculate  and accumulate loss
      y = y.unsqueeze(1).float()
      loss = loss_fn(y_pred, y)
      train_loss += loss.item() 

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      # y_pred_class = y_pred.round()
      # train_acc += (y_pred_class == y).sum().item()/len(y_pred)
      # train_acc += (y_pred_class.eq(y).float().sum().item() / len(y))
      train_acc += ((torch.round(torch.sigmoid(y_pred))).eq(y)).float().mean().item()
      # print(f"Batch: {batch}, Predicted: {y_pred_class}, y_train: {y}")


  # Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  
  # Put model in eval mode
  model.eval() 

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          # test_pred_logits = torch.sigmoid(model(X))
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          y = y.unsqueeze(1).float()
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          # test_pred_labels = test_pred_logits.round()
          # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
          # print((test_pred_logits > 0.5).eq(y).sum())
          test_acc += ((torch.round(torch.sigmoid(test_pred_logits))).eq(y).float().mean().item())

  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

In [18]:
%%time

# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Setup training and save the results
results =   train(model=model,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    epochs=369,
                    device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/369 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.8713 | train_acc: 0.6245 | test_loss: 131.4703 | test_acc: 0.0000
Epoch: 2 | train_loss: 1.0261 | train_acc: 0.7409 | test_loss: 2.1733 | test_acc: 0.0000
Epoch: 3 | train_loss: 0.8268 | train_acc: 0.7867 | test_loss: 141520.0156 | test_acc: 0.0000
Epoch: 4 | train_loss: 0.6587 | train_acc: 0.8164 | test_loss: 557.2786 | test_acc: 0.0000
Epoch: 5 | train_loss: 0.6363 | train_acc: 0.8257 | test_loss: 25001.3359 | test_acc: 0.0000
Epoch: 6 | train_loss: 0.5832 | train_acc: 0.8395 | test_loss: 832493.7500 | test_acc: 0.0000
Epoch: 7 | train_loss: 0.5695 | train_acc: 0.8376 | test_loss: 86360.6172 | test_acc: 0.0000
Epoch: 8 | train_loss: 0.4967 | train_acc: 0.8433 | test_loss: 25699.2207 | test_acc: 0.0000
Epoch: 9 | train_loss: 0.4291 | train_acc: 0.8608 | test_loss: 72096.7656 | test_acc: 0.0000
Epoch: 10 | train_loss: 0.4211 | train_acc: 0.8672 | test_loss: 169772.0938 | test_acc: 0.0000
Epoch: 11 | train_loss: 0.3828 | train_acc: 0.8781 | test_loss: 76524.3828

In [22]:
# Save the trained model to a file
torch.save(model, 'Epoch: 369 | train_loss: 0.0044 | train_acc: 0.9982 | test_loss: 21592.3613 | test_acc: 0.0000.pt')